In [1]:
import wget, os, bz2
import multiprocessing as mp
AP_counts = [60, 126]
counts = sum(AP_counts)
AP_sign = ["", "2"]

In [2]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
folder = "ssds/"
create_folder(folder)

In [4]:
AP_name = "sqlApogee{}Object_{:04d}.csv.bz2"

for ii in range(2):
    for i in range(1, AP_counts[ii]+1):
        name = AP_name.format(AP_sign[ii], i)
        url = "https://dr17.sdss.org/sas/dr17/casload/apCSV/target/" + name
        download = True
        file = folder + name
        if os.path.exists(file):
            download = False
        if download:
            wget.download(url, out=file)
        print("\nFinished downloading {}".format(file))


Finished downloading ssds/sqlApogeeObject_0001.csv.bz2

Finished downloading ssds/sqlApogeeObject_0002.csv.bz2

Finished downloading ssds/sqlApogeeObject_0003.csv.bz2

Finished downloading ssds/sqlApogeeObject_0004.csv.bz2

Finished downloading ssds/sqlApogeeObject_0005.csv.bz2

Finished downloading ssds/sqlApogeeObject_0006.csv.bz2

Finished downloading ssds/sqlApogeeObject_0007.csv.bz2

Finished downloading ssds/sqlApogeeObject_0008.csv.bz2

Finished downloading ssds/sqlApogeeObject_0009.csv.bz2

Finished downloading ssds/sqlApogeeObject_0010.csv.bz2

Finished downloading ssds/sqlApogeeObject_0011.csv.bz2

Finished downloading ssds/sqlApogeeObject_0012.csv.bz2

Finished downloading ssds/sqlApogeeObject_0013.csv.bz2

Finished downloading ssds/sqlApogeeObject_0014.csv.bz2

Finished downloading ssds/sqlApogeeObject_0015.csv.bz2

Finished downloading ssds/sqlApogeeObject_0016.csv.bz2

Finished downloading ssds/sqlApogeeObject_0017.csv.bz2

Finished downloading ssds/sqlApogeeObject_0018.

In [5]:
config = {}
config_file = open("../config.txt", 'r')
for l in config_file:
    l = l.strip()
    if len(l) > 0 and l[0] != "#":
        key, value = [x.strip() for x in l.split("=")]
        config[key] = value

import psycopg
table_name = "ssds"

# Connect to an existing database
with psycopg.connect("dbname={} host={} port={} user={} password={}".format(config["database"], config["hostname"], config["port"], config["username"], config["password"])) as conn:
    with conn.cursor() as cur:
        cur.execute("""DROP TABLE IF EXISTS {}""".format(table_name))
        cur.execute("""
        CREATE TABLE IF NOT EXISTS {} (
            {} BIGINT,
            j DOUBLE PRECISION,
            h DOUBLE PRECISION,
            k DOUBLE PRECISION,
            tmass_prox DOUBLE PRECISION)
        """.format(table_name, config["id_column"]))
        conn.commit()

In [6]:
global_size = mp.Value("i", 0)
global_index = mp.Value("i", 0)
def generating_data(global_size, global_index):
    while True:
        local_index = -1
        ap_version, ap_index = -1, -1
        with global_index.get_lock():
            local_index = global_index.value
            global_index.value += 1
        if local_index >= counts:
            break
        for count in AP_counts:
            ap_version += 1
            if local_index < count:
                ap_index = local_index + 1
            else:
                local_index -= count
        cols = ["j", "h", "k", "tmass_prox"]
        file_name = folder + AP_name.format(AP_sign[ap_version], ap_index)
        f = bz2.open(file_name, mode='rt')
        size = -1
        col_indices = [-1 for i in range(len(cols))]
        data = []
        for l in f:
            tokens = l.split(',')
            if size == -1:
                for i, col in enumerate(cols):
                    for j, token in enumerate(tokens):
                        if col == token:
                            col_indices[i] = j
            else:
                vals = [float(tokens[j]) for j in col_indices]
                if min(vals) >= 0:
                    data.append(vals)
            size += 1
        f.close()
        start_index = -1
        with global_size.get_lock():
            start_index = global_size.value+1
            global_size.value += size
            print("File:", file_name, "Start_index:", start_index, "Size:", size)
        with psycopg.connect("dbname={} host={} port={} user={} password={}".format(config["database"], config["hostname"], config["port"], config["username"], config["password"])) as conn:
            with conn.cursor() as cur:
                with cur.copy("COPY ssds ({}, {}) FROM STDIN".format(config["id_column"], ','.join(cols))) as copy:
                    for i, row in enumerate(data):
                        copy.write_row([start_index + i] + row)
            conn.commit()
        print("File:", file_name, "finished inserting", size, "rows")

In [7]:
ps = [mp.Process(target = generating_data, args = (global_size, global_index)) for i in range(int(config["physical_core"]))]
for p in ps:
    p.start()
for p in ps:
    p.join()

File: ssds/sqlApogee2Object_0018.csv.bz2 Start_index: 1 Size: 1000006
File: ssds/sqlApogee2Object_0031.csv.bz2 Start_index: 1000007 Size: 1000011
File: ssds/sqlApogee2Object_0022.csv.bz2 Start_index: 2000018 Size: 1000029
File: ssds/sqlApogee2Object_0011.csv.bz2 Start_index: 3000047 Size: 1000005
File: ssds/sqlApogee2Object_0051.csv.bz2 Start_index: 4000052 Size: 1000006
File: ssds/sqlApogee2Object_0026.csv.bz2 Start_index: 5000058 Size: 1000029
File: ssds/sqlApogee2Object_0060.csv.bz2 Start_index: 6000087 Size: 1000012
File: ssds/sqlApogee2Object_0021.csv.bz2 Start_index: 7000099 Size: 1000009
File: ssds/sqlApogee2Object_0050.csv.bz2 Start_index: 8000108 Size: 1000000
File: ssds/sqlApogee2Object_0043.csv.bz2 Start_index: 9000108 Size: 1000030
File: ssds/sqlApogee2Object_0041.csv.bz2 Start_index: 10000138 Size: 1000040
File: ssds/sqlApogee2Object_0034.csv.bz2 Start_index: 11000178 Size: 1000039
File: ssds/sqlApogee2Object_0059.csv.bz2 Start_index: 12000217 Size: 1000027
File: ssds/sqlA

File: ssds/sqlApogee2Object_0047.csv.bz2File:  finished insertingssds/sqlApogee2Object_0014.csv.bz2  1000010finished inserting  rows1000015
 rows
File: ssds/sqlApogee2Object_0004.csv.bz2 finished inserting 1000048 rows
File: File:ssds/sqlApogee2Object_0055.csv.bz2 ssds/sqlApogee2Object_0057.csv.bz2  finished insertingfinished inserting  10000281000020  rowsrows

File: ssds/sqlApogee2Object_0056.csv.bz2 finished inserting 1000036 rows
File: ssds/sqlApogee2Object_0025.csv.bz2 finished inserting 1000048 rows
File: ssds/sqlApogee2Object_0016.csv.bz2 finished inserting 1000029 rows
File: ssds/sqlApogee2Object_0020.csv.bz2 finished inserting 1000009 rows
File: ssds/sqlApogee2Object_0035.csv.bz2 finished inserting 1000003 rows
File: ssds/sqlApogee2Object_0015.csv.bz2 finished inserting 1000046 rows
File: ssds/sqlApogee2Object_0052.csv.bz2 finished inserting 1000026 rows
File: ssds/sqlApogee2Object_0044.csv.bz2 finished inserting 1000040 rows
File: ssds/sqlApogee2Object_0028.csv.bz2 finished i

File: ssds/sqlApogee2Object_0081.csv.bz2 Start_index: 131003389 Size: 1000039
File: ssds/sqlApogee2Object_0025.csv.bz2 finished insertingFile:  ssds/sqlApogee2Object_0084.csv.bz21000048  Start_index:rows 
132003428 Size: 1000030
File: ssds/sqlApogee2Object_0056.csv.bz2 Start_index: 133003458 Size: 1000036
File: ssds/sqlApogee2Object_0094.csv.bz2 Start_index: 134003494 Size: 1000017
File: ssds/sqlApogee2Object_0047.csv.bz2 Start_index: 135003511 Size: 1000010
File: ssds/sqlApogee2Object_0031.csv.bz2 finished inserting 1000011 rows
File: ssds/sqlApogee2Object_0096.csv.bz2 Start_index: 136003521 Size: 1000031
File: ssds/sqlApogee2Object_0086.csv.bz2 Start_index: 137003552 Size: 1000021
File: ssds/sqlApogee2Object_0089.csv.bz2 Start_index: 138003573 Size: 1000012
File: ssds/sqlApogee2Object_0088.csv.bz2 Start_index: 139003585 Size: 1000032
File: ssds/sqlApogee2Object_0069.csv.bz2 Start_index: 140003617 Size: 1000042
File: ssds/sqlApogee2Object_0085.csv.bz2 Start_index: 141003659 Size: 1000

File: ssds/sqlApogee2Object_0126.csv.bz2 Start_index: 165004201 Size: 661641
File: ssds/sqlApogee2Object_0104.csv.bz2 Start_index: 165665842 Size: 1000031
File: ssds/sqlApogee2Object_0106.csv.bz2 Start_index: 166665873 Size: 1000006
File: ssds/sqlApogee2Object_0112.csv.bz2 Start_index: 167665879 Size: 1000041
File: ssds/sqlApogee2Object_0117.csv.bz2 Start_index: 168665920 Size: 1000023
File: ssds/sqlApogee2Object_0116.csv.bz2 Start_index: 169665943 Size: 1000046
File: ssds/sqlApogee2Object_0109.csv.bz2 Start_index: 170665989 Size: 1000010
File: ssds/sqlApogee2Object_0120.csv.bz2 Start_index: 171665999 Size: 1000040
File: ssds/sqlApogee2Object_0111.csv.bz2 Start_index: 172666039 Size: 1000012
File: ssds/sqlApogee2Object_0113.csv.bz2 Start_index: 173666051 Size: 1000028
File: ssds/sqlApogee2Object_0119.csv.bz2 Start_index: 174666079 Size: 1000022
File: ssds/sqlApogee2Object_0110.csv.bz2 Start_index: 175666101 Size: 1000041
File: ssds/sqlApogee2Object_0118.csv.bz2 Start_index: 176666142 S

In [8]:
print("Starts indexing ssds")
with psycopg.connect("dbname={} host={} port={} user={} password={}".format(config["database"], config["hostname"], config["port"], config["username"], config["password"])) as conn:
    with conn.cursor() as cur:
        cur.execute("""ALTER TABLE {} ADD PRIMARY KEY ({})""".format(table_name, config["id_column"]))
        conn.commit()
print("Finished indexing ssds")

Starts indexing ssds
Finished indexing ssds
